In [1]:
import random
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [2]:
#Set TF seed for reproducability
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

In [3]:
#Data Load
dataf = pd.read_csv("../Datasets/MVPdata_final.csv")

In [4]:
#data filter
df = dataf[((dataf['G']>48)&(dataf['PTS']>13.5)&(dataf['MP']>30)
           &(dataf['AST']>1)&(dataf['TRB']>3)&(dataf['FG%']>0.37))]
df['Odds'] = -df['Odds']

C:\Users\xx049\AppData\Local\Temp\ipykernel_20452\2188782804.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Odds'] = -df['Odds']


In [5]:
#years to be changed for backtest
dfpred = df[df['Season'] == 2022]
df = df[(df['Season'] >= 2007) & (df['Season'] <= 2021)]

In [6]:
#Feature Selections
features =  ['AST', 'STL', 'G', 'PTS', 'WS', 'eFG%', 'Raptor Offense','Raptor Defense','Predator Defense', 'Odds']
X = df[features]

In [7]:
y = df['Share']


In [8]:
#Dataset Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Define correlation coefficients
correlation_coeffs = {
    'AST': 0.254608,
    'STL': 0.221764,
    'G': 0.155084,
    'PTS': 0.471422,
    'WS': 0.644362,
    'eFG%': 0.284468,
    'Raptor Offense': 0.327580,
    'Raptor Defense': 0.106614,
    'Predator Defense': 0.117243,
    'Odds': 0.312252,}
correlation_threshold = 0

#Features with correlation values above the threshold
selected_features = [feature for feature, coeff in correlation_coeffs.items() if abs(coeff) > correlation_threshold]

#DataFrame containing the selected features and their correlation coefficients
corr_df = pd.DataFrame({
    'Feature': selected_features,
    'Correlation_Coeff': [correlation_coeffs[feature] for feature in selected_features]
})

#Standardize
scaler = StandardScaler()
corr_df['Correlation_Coeff_Scaled'] = scaler.fit_transform(corr_df[['Correlation_Coeff']])

# Merge the correlation coefficient feature with your original dataset
df_with_corr_features = pd.concat([df, corr_df.set_index('Feature')], axis=1, join='inner')

# Extract the selected features for X_train and X_test
y_train_reshaped = y_train.values.reshape(-1, 1)
y_test_reshaped = y_test.values.reshape(-1, 1)

X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]
y_train_scaled = scaler.transform(y_train_reshaped)
y_test_scaled = scaler.transform(y_test_reshaped)

C:\Users\xx049\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\xx049\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [10]:
#neural network model
num_selected_features = len(selected_features)

model = Sequential()
input_shape = (num_selected_features,)

model.add(layers.Input(shape=input_shape))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1, activation='linear'))
model.compile(optimizer='adam', loss='mean_squared_error')

In [11]:
model.fit(X_train_selected, y_train_scaled, epochs=1000, batch_size=32, verbose=1)

Epoch 1/1000
5/5 [==============================] - 1s 3ms/step - loss: 18846852.0000
Epoch 2/1000
5/5 [==============================] - 0s 3ms/step - loss: 11292623.0000
Epoch 3/1000
5/5 [==============================] - 0s 3ms/step - loss: 4187944.5000
Epoch 4/1000
5/5 [==============================] - 0s 3ms/step - loss: 5982252.5000
Epoch 5/1000
5/5 [==============================] - 0s 2ms/step - loss: 6953290.0000
Epoch 6/1000
5/5 [==============================] - 0s 2ms/step - loss: 3331485.2500
Epoch 7/1000
5/5 [==============================] - 0s 2ms/step - loss: 2547570.0000
Epoch 8/1000
5/5 [==============================] - 0s 2ms/step - loss: 2634397.2500
Epoch 9/1000
5/5 [==============================] - 0s 2ms/step - loss: 3715784.5000
Epoch 10/1000
5/5 [==============================] - 0s 3ms/step - loss: 2140272.5000
Epoch 11/1000
5/5 [==============================] - 0s 4ms/step - loss: 1223232.2500
Epoch 12/1000
5/5 [==============================] - 0s 3ms/s

5/5 [==============================] - 0s 2ms/step - loss: 111282.0391
Epoch 98/1000
5/5 [==============================] - 0s 2ms/step - loss: 119744.2500
Epoch 99/1000
5/5 [==============================] - 0s 2ms/step - loss: 104439.4766
Epoch 100/1000
5/5 [==============================] - 0s 2ms/step - loss: 90195.0625
Epoch 101/1000
5/5 [==============================] - 0s 2ms/step - loss: 118833.8359
Epoch 102/1000
5/5 [==============================] - 0s 2ms/step - loss: 135199.2500
Epoch 103/1000
5/5 [==============================] - 0s 2ms/step - loss: 50903.8047
Epoch 104/1000
5/5 [==============================] - 0s 2ms/step - loss: 134427.0000
Epoch 105/1000
5/5 [==============================] - 0s 2ms/step - loss: 73077.4219
Epoch 106/1000
5/5 [==============================] - 0s 2ms/step - loss: 147525.6094
Epoch 107/1000
5/5 [==============================] - 0s 2ms/step - loss: 99415.5859
Epoch 108/1000
5/5 [==============================] - 0s 2ms/step - loss: 7

5/5 [==============================] - 0s 2ms/step - loss: 13146.0752
Epoch 194/1000
5/5 [==============================] - 0s 2ms/step - loss: 52182.4297
Epoch 195/1000
5/5 [==============================] - 0s 2ms/step - loss: 46327.7539
Epoch 196/1000
5/5 [==============================] - 0s 2ms/step - loss: 40908.6953
Epoch 197/1000
5/5 [==============================] - 0s 2ms/step - loss: 19730.8906
Epoch 198/1000
5/5 [==============================] - 0s 2ms/step - loss: 25857.1406
Epoch 199/1000
5/5 [==============================] - 0s 2ms/step - loss: 31943.6094
Epoch 200/1000
5/5 [==============================] - 0s 2ms/step - loss: 31763.9922
Epoch 201/1000
5/5 [==============================] - 0s 2ms/step - loss: 13022.2236
Epoch 202/1000
5/5 [==============================] - 0s 2ms/step - loss: 33612.5664
Epoch 203/1000
5/5 [==============================] - 0s 2ms/step - loss: 17515.4707
Epoch 204/1000
5/5 [==============================] - 0s 2ms/step - loss: 70169.

5/5 [==============================] - 0s 2ms/step - loss: 6558.2041
Epoch 290/1000
5/5 [==============================] - 0s 2ms/step - loss: 25851.2539
Epoch 291/1000
5/5 [==============================] - 0s 2ms/step - loss: 29764.5508
Epoch 292/1000
5/5 [==============================] - 0s 2ms/step - loss: 59413.8828
Epoch 293/1000
5/5 [==============================] - 0s 2ms/step - loss: 55016.8438
Epoch 294/1000
5/5 [==============================] - 0s 2ms/step - loss: 3692.2952
Epoch 295/1000
5/5 [==============================] - 0s 2ms/step - loss: 40487.5586
Epoch 296/1000
5/5 [==============================] - 0s 2ms/step - loss: 10992.0811
Epoch 297/1000
5/5 [==============================] - 0s 2ms/step - loss: 7412.5786
Epoch 298/1000
5/5 [==============================] - 0s 2ms/step - loss: 5208.5938
Epoch 299/1000
5/5 [==============================] - 0s 2ms/step - loss: 9093.8193
Epoch 300/1000
5/5 [==============================] - 0s 2ms/step - loss: 5534.6567
E

5/5 [==============================] - 0s 2ms/step - loss: 4805.0396
Epoch 387/1000
5/5 [==============================] - 0s 2ms/step - loss: 9300.1035
Epoch 388/1000
5/5 [==============================] - 0s 2ms/step - loss: 4863.9019
Epoch 389/1000
5/5 [==============================] - 0s 2ms/step - loss: 7401.5190
Epoch 390/1000
5/5 [==============================] - 0s 2ms/step - loss: 6877.1011
Epoch 391/1000
5/5 [==============================] - 0s 2ms/step - loss: 8556.3984
Epoch 392/1000
5/5 [==============================] - 0s 2ms/step - loss: 6897.9058
Epoch 393/1000
5/5 [==============================] - 0s 2ms/step - loss: 1801.2496
Epoch 394/1000
5/5 [==============================] - 0s 2ms/step - loss: 8025.6431
Epoch 395/1000
5/5 [==============================] - 0s 2ms/step - loss: 33375.2500
Epoch 396/1000
5/5 [==============================] - 0s 2ms/step - loss: 7309.9526
Epoch 397/1000
5/5 [==============================] - 0s 2ms/step - loss: 4096.3516
Epoch 

...

5/5 [==============================] - 0s 2ms/step - loss: 111.6544
Epoch 581/1000
5/5 [==============================] - 0s 2ms/step - loss: 216.1560
Epoch 582/1000
5/5 [==============================] - 0s 2ms/step - loss: 861.9741
Epoch 583/1000
5/5 [==============================] - 0s 2ms/step - loss: 2043.1444
Epoch 584/1000
5/5 [==============================] - 0s 2ms/step - loss: 978.6075
Epoch 585/1000
5/5 [==============================] - 0s 2ms/step - loss: 1244.5271
Epoch 586/1000
5/5 [==============================] - 0s 2ms/step - loss: 388.2208
Epoch 587/1000
5/5 [==============================] - 0s 1ms/step - loss: 1913.6573
Epoch 588/1000
5/5 [==============================] - 0s 2ms/step - loss: 614.0746
Epoch 589/1000
5/5 [==============================] - 0s 2ms/step - loss: 337.4270
Epoch 590/1000
5/5 [==============================] - 0s 2ms/step - loss: 2667.6760
Epoch 591/1000
5/5 [==============================] - 0s 2ms/step - loss: 20390.0039
Epoch 592/100

Epoch 678/1000
5/5 [==============================] - 0s 2ms/step - loss: 899.0411
Epoch 679/1000
5/5 [==============================] - 0s 2ms/step - loss: 2904.0510
Epoch 680/1000
5/5 [==============================] - 0s 2ms/step - loss: 453.1022
Epoch 681/1000
5/5 [==============================] - 0s 2ms/step - loss: 2950.4937
Epoch 682/1000
5/5 [==============================] - 0s 2ms/step - loss: 453.3562
Epoch 683/1000
5/5 [==============================] - 0s 2ms/step - loss: 2747.4307
Epoch 684/1000
5/5 [==============================] - 0s 2ms/step - loss: 447.2063
Epoch 685/1000
5/5 [==============================] - 0s 2ms/step - loss: 300.1012
Epoch 686/1000
5/5 [==============================] - 0s 2ms/step - loss: 2325.1829
Epoch 687/1000
5/5 [==============================] - 0s 1ms/step - loss: 3271.2397
Epoch 688/1000
5/5 [==============================] - 0s 2ms/step - loss: 1601.6674
Epoch 689/1000
5/5 [==============================] - 0s 2ms/step - loss: 492.700

5/5 [==============================] - 0s 2ms/step - loss: 661.7518
Epoch 777/1000
5/5 [==============================] - 0s 2ms/step - loss: 491.9726
Epoch 778/1000
5/5 [==============================] - 0s 2ms/step - loss: 1750.1910
Epoch 779/1000
5/5 [==============================] - 0s 2ms/step - loss: 2156.5469
Epoch 780/1000
5/5 [==============================] - 0s 2ms/step - loss: 189.8329
Epoch 781/1000
5/5 [==============================] - 0s 2ms/step - loss: 763.8431
Epoch 782/1000
5/5 [==============================] - 0s 2ms/step - loss: 12900.9521
Epoch 783/1000
5/5 [==============================] - 0s 2ms/step - loss: 1979.9229
Epoch 784/1000
5/5 [==============================] - 0s 2ms/step - loss: 72.8191
Epoch 785/1000
5/5 [==============================] - 0s 2ms/step - loss: 1196.6907
Epoch 786/1000
5/5 [==============================] - 0s 2ms/step - loss: 1333.1110
Epoch 787/1000
5/5 [==============================] - 0s 2ms/step - loss: 116.7372
Epoch 788/100

5/5 [==============================] - 0s 1ms/step - loss: 883.5117
Epoch 875/1000
5/5 [==============================] - 0s 2ms/step - loss: 1555.8517
Epoch 876/1000
5/5 [==============================] - 0s 2ms/step - loss: 36.0504
Epoch 877/1000
5/5 [==============================] - 0s 2ms/step - loss: 378.8924
Epoch 878/1000
5/5 [==============================] - 0s 2ms/step - loss: 1509.4976
Epoch 879/1000
5/5 [==============================] - 0s 2ms/step - loss: 1113.1044
Epoch 880/1000
5/5 [==============================] - 0s 2ms/step - loss: 229.5213
Epoch 881/1000
5/5 [==============================] - 0s 2ms/step - loss: 384.0191
Epoch 882/1000
5/5 [==============================] - 0s 2ms/step - loss: 704.2428
Epoch 883/1000
5/5 [==============================] - 0s 2ms/step - loss: 1776.2673
Epoch 884/1000
5/5 [==============================] - 0s 2ms/step - loss: 440.2260
Epoch 885/1000
5/5 [==============================] - 0s 2ms/step - loss: 3180.1201
Epoch 886/1000


5/5 [==============================] - 0s 2ms/step - loss: 375.9421
Epoch 973/1000
5/5 [==============================] - 0s 3ms/step - loss: 24.1069
Epoch 974/1000
5/5 [==============================] - 0s 2ms/step - loss: 1292.9598
Epoch 975/1000
5/5 [==============================] - 0s 2ms/step - loss: 1697.7338
Epoch 976/1000
5/5 [==============================] - 0s 2ms/step - loss: 62.9492
Epoch 977/1000
5/5 [==============================] - 0s 2ms/step - loss: 73.5943
Epoch 978/1000
5/5 [==============================] - 0s 2ms/step - loss: 53.9677
Epoch 979/1000
5/5 [==============================] - 0s 2ms/step - loss: 108.0715
Epoch 980/1000
5/5 [==============================] - 0s 2ms/step - loss: 67.1136
Epoch 981/1000
5/5 [==============================] - 0s 2ms/step - loss: 785.7540
Epoch 982/1000
5/5 [==============================] - 0s 2ms/step - loss: 88.0510
Epoch 983/1000
5/5 [==============================] - 0s 2ms/step - loss: 213.0379
Epoch 984/1000
5/5 [===

In [12]:
#Evaluate
y_pred = model.predict(X_test_selected)
mse = mean_squared_error(y_test_scaled, y_pred)
print(f"Mean Squared Error: {mse:.2f}")

2/2 [==============================] - 0s 2ms/step
Mean Squared Error: 4.71


In [13]:
scaler = StandardScaler()
scaler.fit(X_train_selected)

# Transform
X_train_scaled = scaler.transform(X_train_selected)

# Select and transform only the relevant features
X_pred_selected = dfpred[selected_features]
X_pred_scaled = scaler.transform(X_pred_selected)

In [14]:
y_pred = model.predict(X_pred_scaled)

1/1 [==============================] - 0s 55ms/step


In [15]:
results_df = pd.DataFrame({
    'Predicted MVP': dfpred['Player'],
    'MVP Share Prediction': y_pred.flatten(),
    'Actual Share': dfpred['Share'],
    'Actual Placement': dfpred['Rank'],
})

In [16]:
results_df

,Predicted MVP,MVP Share Prediction,Actual Share,Actual Placement
208,Nikola Jokic,0.104481,0.875,1
209,Joel Embiid,0.121943,0.706,2
210,Giannis Antetokounmpo,0.102579,0.595,3
211,Devin Booker,0.113106,0.216,4
212,Luka Doncic,0.106688,0.146,5
213,Jayson Tatum,0.113106,0.043,6
214,Ja Morant,0.098302,0.010,7
215,Stephen Curry,0.111481,0.004,8
216,Chris Paul,0.115034,0.002,9
217,DeMar DeRozan,0.113532,0.001,10T


In [17]:
results_df = results_df.sort_values(by='MVP Share Prediction', ascending=False)
winner = results_df.head(5)
print(winner)

     Predicted MVP  MVP Share Prediction  Actual Share Actual Placement
209    Joel Embiid              0.121943         0.706                2
216     Chris Paul              0.115034         0.002                9
217  DeMar DeRozan              0.113532         0.001              10T
211   Devin Booker              0.113106         0.216                4
213   Jayson Tatum              0.113106         0.043                6
